In [5]:
import pandas as pd


In [6]:
df_sarc = pd.read_csv("train-balanced-sarcasm.csv")


In [7]:
df_sarc = pd.read_csv("train-balanced-sarcasm.csv")


In [8]:
df_sarc.head()


,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [9]:
df_sarc.shape


(1010826, 10)

In [10]:
df_sarc["label"].value_counts()


label
0    505413
1    505413
Name: count, dtype: int64

In [12]:
df_sarc.columns


Index(['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs',
       'date', 'created_utc', 'parent_comment'],
      dtype='object')

In [13]:
df_sarc = df_sarc.rename(columns={
    "comment": "text",
    "label": "sarcasm"
})


In [14]:
df_sarc = df_sarc[["text", "sarcasm"]]
df_sarc.head()


,text,sarcasm
0,NC and NH.,0
1,You do know west teams play against west teams...,0
2,"They were underdogs earlier today, but since G...",0
3,"This meme isn't funny none of the ""new york ni...",0
4,I could use one of those tools.,0


In [15]:
df_sarc["text"] = df_sarc["text"].astype(str)
df_sarc = df_sarc.dropna()


In [16]:
df_sarc.sample(5)


,text,sarcasm
888048,Everyone knows that they're the greatest thing...,1
847288,Por que no los dos?,0
961275,The refs are giving this game to the Thunder.,1
982195,It was a unit joke... cm vs m .... No offense,0
787865,Point Reyes isn't in the OC!,1


In [17]:
df_sarc = df_sarc.rename(columns={
    "comment": "text",
    "label": "sarcasm"
})

df_sarc.head()


,text,sarcasm
0,NC and NH.,0
1,You do know west teams play against west teams...,0
2,"They were underdogs earlier today, but since G...",0
3,"This meme isn't funny none of the ""new york ni...",0
4,I could use one of those tools.,0


In [18]:
df_sarc.isnull().sum()


text       0
sarcasm    0
dtype: int64

In [23]:
len(X_train), len(X_test)


(808660, 202166)

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assume df_sarc already loaded
X = df_sarc["text"].astype(str).tolist()
y = df_sarc["sarcasm"].tolist()   # 0 = non-sarcasm, 1 = sarcasm

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [37]:
len(X_train), len(X_test)


(808660, 202166)

In [38]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    X_test,
    truncation=True,
    padding=True,
    max_length=128
)


In [39]:
type(train_encodings)
train_encodings.keys()


In [40]:
import torch

class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SarcasmDataset(train_encodings, y_train)
eval_dataset = SarcasmDataset(test_encodings, y_test)


In [41]:
len(train_dataset)
train_dataset[0]


{'input_ids': tensor([ 101, 2430, 4307,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0

In [43]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

NameError: name 'DistilBertForSequenceClassification' is not defined

In [44]:
from transformers import DistilBertForSequenceClassification


In [45]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
# Training config (CPU-friendly)
training_args = TrainingArguments(
    output_dir="./sarcasm_results",
    max_steps=300,           # same as before
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=50,
    save_strategy="no",
    report_to="none",
    use_cpu=True
)

NameError: name 'TrainingArguments' is not defined

In [47]:
from transformers import TrainingArguments


In [48]:
# Training config (CPU-friendly)
training_args = TrainingArguments(
    output_dir="./sarcasm_results",
    max_steps=300,           # same as before
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=50,
    save_strategy="no",
    report_to="none",
    use_cpu=True
)

In [49]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

NameError: name 'Trainer' is not defined

In [50]:
from transformers import Trainer


In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [52]:
trainer.train()

Step,Training Loss
50,0.695600
100,0.689300
150,0.667400
200,0.611300
250,0.630600
300,0.675100


TrainOutput(global_step=300, training_loss=0.6615681521097819, metrics={'train_runtime': 567.5929, 'train_samples_per_second': 4.228, 'train_steps_per_second': 0.529, 'total_flos': 79480439193600.0, 'train_loss': 0.6615681521097819, 'epoch': 0.0029678580968115313})

In [53]:
model.save_pretrained("distilbert_sarcasm_model")
tokenizer.save_pretrained("distilbert_sarcasm_model")


('distilbert_sarcasm_model\\tokenizer_config.json',
 'distilbert_sarcasm_model\\special_tokens_map.json',
 'distilbert_sarcasm_model\\vocab.txt',
 'distilbert_sarcasm_model\\added_tokens.json',
 'distilbert_sarcasm_model\\tokenizer.json')

In [2]:
# ===============================
# A. IMPORTS (must re-run)
# ===============================
import torch
import pandas as pd
import numpy as np

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

from sklearn.model_selection import train_test_split


In [3]:
# ===============================
# B. LOAD SARCASTIC DATASET
# ===============================
# Example file name – adjust if different
df = pd.read_csv("train-balanced-sarcasm.csv")

# Keep only needed columns
df = df[["comment", "label"]]
df = df.dropna()

df = df.rename(columns={
    "comment": "text",
    "label": "sarcasm"
})

df.head()


,text,sarcasm
0,NC and NH.,0
1,You do know west teams play against west teams...,0
2,"They were underdogs earlier today, but since G...",0
3,"This meme isn't funny none of the ""new york ni...",0
4,I could use one of those tools.,0


In [4]:
# ===============================
# B. LOAD SARCASTIC DATASET
# ===============================
# Example file name – adjust if different
df = pd.read_csv("train-balanced-sarcasm.csv")

# Keep only needed columns
df = df[["comment", "label"]]
df = df.dropna()

df = df.rename(columns={
    "comment": "text",
    "label": "sarcasm"
})

df.head()


,text,sarcasm
0,NC and NH.,0
1,You do know west teams play against west teams...,0
2,"They were underdogs earlier today, but since G...",0
3,"This meme isn't funny none of the ""new york ni...",0
4,I could use one of those tools.,0


In [5]:
# ===============================
# C. TRAIN / TEST SPLIT
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    df["text"].tolist(),
    df["sarcasm"].tolist(),
    test_size=0.2,
    random_state=42
)


In [6]:
# ===============================
# D. TOKENIZER
# ===============================
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased"
)

train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    X_test,
    truncation=True,
    padding=True,
    max_length=128
)


In [7]:
# ===============================
# E. TORCH DATASET
# ===============================
class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SarcasmDataset(train_encodings, y_train)
eval_dataset = SarcasmDataset(test_encodings, y_test)


In [8]:
# ===============================
# F. LOAD MODEL (CONTINUE LEARNING)
# ===============================
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [9]:
# ===============================
# G. TRAINING ARGS (NEXT +300 STEPS)
# ===============================
training_args = TrainingArguments(
    output_dir="./sarcasm_results",
    max_steps=600,              # 🔥 CONTINUE to 600
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=50,
    save_strategy="no",
    report_to="none",
    use_cpu=True
)


In [10]:
# ===============================
# H. TRAINER
# ===============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)


In [11]:
# ===============================
# I. TRAIN (THIS IS THE +300)
# ===============================
trainer.train()


Step,Training Loss
50,0.694000
100,0.683100
150,0.663300
200,0.665200
250,0.655700
300,0.628600
350,0.603500
400,0.630200
450,0.628300
500,0.614700


TrainOutput(global_step=600, training_loss=0.6381821727752686, metrics={'train_runtime': 1048.8544, 'train_samples_per_second': 4.576, 'train_steps_per_second': 0.572, 'total_flos': 158960878387200.0, 'train_loss': 0.6381821727752686, 'epoch': 0.005936068541804763})

In [12]:
# ===============================
# J. SAVE MODEL (IMPORTANT)
# ===============================
model.save_pretrained("distilbert_sarcasm_model")
tokenizer.save_pretrained("distilbert_sarcasm_model")


('distilbert_sarcasm_model\\tokenizer_config.json',
 'distilbert_sarcasm_model\\special_tokens_map.json',
 'distilbert_sarcasm_model\\vocab.txt',
 'distilbert_sarcasm_model\\added_tokens.json',
 'distilbert_sarcasm_model\\tokenizer.json')